In [66]:
# Import Data 
from quantopian.pipeline import Pipeline, CustomFactor
from quantopian.pipeline.data import EquityPricing
from quantopian.pipeline.domain import US_EQUITIES
from quantopian.research import run_pipeline
from quantopian.pipeline.factors import SimpleMovingAverage

import numpy as np

# This is used to inform pipeline engine which column to retrieve for computations
daily_close = EquityPricing.close.latest
daily_volume = EquityPricing.volume.latest

# Define Computations 
pipe = Pipeline(
    columns = {
        'close':daily_close,
        'volume':daily_volume
    },
    domain=US_EQUITIES
)


# Instantiate a Pipeline 
df = run_pipeline(pipe, '2010-01-01','2011-12-31')
print(df.tail())

                                                    close    volume
2012-01-03 00:00:00+00:00 Equity(42306 [SMH])       30.43  629754.0
                          Equity(42315 [RSE])       12.74  351047.0
                          Equity(42316 [GGP_WI])    14.65     100.0
                          Equity(42320 [SCD_WD])      NaN       0.0
                          Equity(42334 [FNFG_PRB])  25.60  126928.0


In [71]:
def initialize(context):
    context.stock = symbol('AAPL')

In [77]:
class StandardDeviation(CustomFactor):
    def compute(self, today, asset_ids, out, values):
        out[:] = np.nanstd(values, axis=0)

In [90]:
def make_pipeline():
    
    # This is a FACTOR, which is like a function. We do not get the result until we add this to our pipe 
    sma_20 = SimpleMovingAverage(
                inputs = [EquityPricing.close],
                window_length = 20
    )
    
    last_price =  EquityPricing.close.latest
    
    price_sma_ratio = last_price/sma_20
    
    price_std = StandardDeviation(
                inputs = [EquityPricing.close],
                window_length = 20
    )
    
    BB_High = sma_20 + 2*price_std
    BB_Low = sma_20 - 2*price_std
    BB_Percent = (last_price - BB_Low) / (BB_High - BB_Low)
    
    return Pipeline(
        columns = {
            "20_Day_SMA": sma_20,
            "Latest_Price": last_price,
            "Price_SMA_Ratio": price_sma_ratio,
            "STD": price_std,
            "BB_High":BB_High,
            "BB_Low":BB_Low,
            "BB_Percent":BB_Percent
        },
    domain=US_EQUITIES
    )

In [91]:
my_pipe = make_pipeline()

In [92]:
result = run_pipeline(my_pipe, '2010-01-01','2011-12-31')

In [93]:
# This will show an index for all our securities over the time period

# This is a Multi-Index dataframe, with date and Symbol in Index
result.head()

20_Day_SMA     BB_High  \
2010-01-04 00:00:00+00:00 Equity(2 [HWM])       14.762950   17.219383   
                          Equity(21 [AAME])      1.248125    1.380454   
                          Equity(24 [AAPL])    199.193000  213.578101   
                          Equity(25 [HWM_PR])   63.622143   65.167645   
                          Equity(31 [ABAX])     23.759000   25.630522   

                                                   BB_Low  BB_Percent  \
2010-01-04 00:00:00+00:00 Equity(2 [HWM])       12.306517    0.776224   
                          Equity(21 [AAME])      1.115796    0.620439   
                          Equity(24 [AAPL])    184.807899    0.904829   
                          Equity(25 [HWM_PR])   62.076640         NaN   
                          Equity(31 [ABAX])     21.887478    0.978487   

                                               Latest_Price  Price_SMA_Ratio  \
2010-01-04 00:00:00+00:00 Equity(2 [HWM])             16.12         1.091923   
                          Equity(21 [AAME])            1.28         1.025538   
                          Equity(24 [AAPL])          210.84         1.058471   
                          Equity(25 [HWM_PR])           NaN              NaN   
                          Equity(31 [ABAX])           25.55         1.075382   

                                                    STD  
2010-01-04 00:00:00+00:00 Equity(2 [HWM])      1.228217  
                          Equity(21 [AAME])    0.066164  
                          Equity(24 [AAPL])    7.192550  
                          Equity(25 [HWM_PR])  0.772751  
                          Equity(31 [ABAX])    0.935761